In [4]:
!pip install qiskit-machine-learning
!pip install qiskit qiskit_machine_learning qiskit_algorithms
!pip install qiskit-aer
!pip install genomic-benchmarks

from IPython.display import clear_output
clear_output()

In [8]:
from qiskit.circuit.library import PauliFeatureMap

In [5]:

from qiskit.circuit.library import PauliFeatureMap
from qiskit.primitives import Sampler
from qiskit_algorithms.state_fidelities import ComputeUncompute
#from qiskit_machine_learning.kernels import FidelityQuantumKernel,QuantumKernel
from qiskit_machine_learning.kernels import FidelityQuantumKernel
import time
#from qiskit import BasicAer#
from qiskit_algorithms.utils import algorithm_globals

algorithm_globals.random_seed = 12345
from qiskit_algorithms.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal
from qiskit_machine_learning.algorithms import VQC
#from qiskit_machine_learning.datasets import iris
from qiskit_machine_learning.circuit.library import RawFeatureVector
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from qiskit_machine_learning.algorithms import QSVC

from sklearn.preprocessing import MinMaxScaler # Importing MinMaxScaler from sklearn.preprocessing
# Variables to track objective function and learning rate
# Set random seed for reproducibility
seed = 1376
algorithm_globals.random_seed = seed

from genomic_benchmarks.dataset_getters.pytorch_datasets import DemoHumanOrWorm

test_set = DemoHumanOrWorm(split='test', version=0)
train_set = DemoHumanOrWorm(split='train', version=0)

data_set = train_set
# data_set = train_set + test_set
len(data_set)
print(f"Number of samples in the test set: {len(test_set)}")
print(f"Number of samples in the test set: {len(train_set)}")
from collections import defaultdict
import numpy as np


# Filter out sequences that consist entirely of "N"
filtered_data_set = [sample for sample in data_set if not all(base == 'N' for base in sample[0])]

print(f"Filtered data set size: {len(filtered_data_set)}")
print("One sample from the filtered data set:")
print(filtered_data_set[0])

word_size = 40
word_combinations = defaultdict(int)
iteration = 1
for text, _ in filtered_data_set:
    for i in range(len(text)):
        word = text[i:i+word_size]
        if word_combinations.get(word) is None:
          word_combinations[word] = iteration
          iteration += 1


print("\nFirst 5 samples in the word_combinations dict.")
for key, value in list(word_combinations.items())[:5]:
    print(key, value)

import numpy as np
# Preprocess the training set
np_data_set = []
for i in range(len(data_set)):
    sequence, label = data_set[i]
    sequence = sequence.strip()  # Remove any leading/trailing whitespace
    words = [sequence[i:i + word_size] for i in range(0, len(sequence), word_size)]  # Split the sequence into 4-letter words
    int_sequence = np.array([word_combinations[word] for word in words])
    data_point = {'sequence': int_sequence, 'label': label}
    np_data_set.append(data_point)


print("First 5 samples of encoded data:")
np_data_set[:5]

np.random.shuffle(np_data_set)
print("First 5 samples of encoded shuffled data:")
np_data_set[:5]


from sklearn.preprocessing import MinMaxScaler

sequences = np.array([item['sequence'] for item in np_data_set])
sequences = np.vstack(sequences)

scaler = MinMaxScaler()

sequences_scaled = scaler.fit_transform(sequences)
from sklearn.decomposition import PCA
# Apply PCA to reduce dimensionality of the scaled data
pca = PCA(n_components=2)  # Reduce to 2 components; adjust this number as needed
sequences_pca = pca.fit_transform(sequences_scaled)

# Update the sequences in np_data_set with the PCA-transformed sequences
for i, item in enumerate(np_data_set):
    item['sequence'] = sequences_pca[i]

# Shuffle the dataset again after transformation
np.random.shuffle(np_data_set)

print("First 5 samples of scaled encoded shuffled data:")
np_data_set[:5]

np_train_data = np_data_set[:200]
np_test_data = np_data_set[-20:]

print(f"Length of np_train_data: {len(np_train_data)}")
print(f"Length of np_test_data: {len(np_test_data)}")

# Extract the PCA-transformed sequences and labels
train_sequences = np.array([data_point["sequence"] for data_point in np_train_data])
train_labels = np.array([data_point["label"] for data_point in np_train_data])

test_sequences = np.array([data_point["sequence"] for data_point in np_test_data])
test_labels = np.array([data_point["label"] for data_point in np_test_data])

print(f"Shape of reduced train_sequences: {train_sequences.shape}")
print(f"Shape of reduced test_sequences: {test_sequences.shape}")

# Setup feature map and ansatz for VQC
num_features = train_sequences.shape[1]
##feature_map = ZZFeatureMap(feature_dimension=num_features, reps=1)
#ansatz = RealAmplitudes(num_qubits=num_features, reps=3)

from sklearn.preprocessing import MinMaxScaler

#features = MinMaxScaler().fit_transform(num_features)
import pandas as pd
import seaborn as sns


from sklearn.model_selection import train_test_split
from qiskit_algorithms.utils import algorithm_globals

algorithm_globals.random_seed = 123
data_train, data_test, data_train_labels, data_test_labels  = train_test_split(
    train_sequences, train_labels, train_size=0.8, random_state=algorithm_globals.random_seed
)



Number of samples in the test set: 25000
Number of samples in the test set: 75000
Filtered data set size: 73118
One sample from the filtered data set:
('TTTCGATTCAAAAATTGCAAAATTGTTTCAATTAAGCTCTTTTGCACCTGAAAGTTTAAAATGTACAAAAGAAAAATGCTAGAATATTCAAGAAAATCGAATTTTCCATCTAAAAATCGCCGCGCCGAGAAGCTTTTGCTCTGTGAGCTCCAAATAGGAAGCCAAAAACTTTCGATTTGAAGCCCAAAGAGCAGCTTTTC', 0)

First 5 samples in the word_combinations dict.
TTTCGATTCAAAAATTGCAAAATTGTTTCAATTAAGCTCT 1
TTCGATTCAAAAATTGCAAAATTGTTTCAATTAAGCTCTT 2
TCGATTCAAAAATTGCAAAATTGTTTCAATTAAGCTCTTT 3
CGATTCAAAAATTGCAAAATTGTTTCAATTAAGCTCTTTT 4
GATTCAAAAATTGCAAAATTGTTTCAATTAAGCTCTTTTG 5
First 5 samples of encoded data:
First 5 samples of encoded shuffled data:
First 5 samples of scaled encoded shuffled data:
Length of np_train_data: 200
Length of np_test_data: 20
Shape of reduced train_sequences: (200, 2)
Shape of reduced test_sequences: (20, 2)


This code provide QSVC defualt vsualization using featuremap

In [10]:

objective_func_vals = []
learning_rates = []
perturbations = []
initial_learning_rate = 0.15  # Starting learning rate
initial_perturbation = 0.15  # Starting perturbation value
momentum = 0.95  # Momentum factor for smoother learning rate adjustments
gradient_moving_avg = 0  # Moving average of gradients


# Define quantum feature map and ansatz
feature_dim = data_train.shape[1]

import matplotlib.pyplot as plt
import numpy as np

feature_map = PauliFeatureMap(feature_dimension=feature_dim, reps=3)

sampler = Sampler()

fidelity = ComputeUncompute(sampler=sampler)

kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=feature_map)
print(feature_map.num_qubits)

qsvc=QSVC(quantum_kernel=kernel)
t0=time.time()
qsvc.fit(data_train, data_train_labels)
t1 = time.time()
print(f"Time taken for QSVC training PauliFeatureMap: {t1 - t0} seconds")

qsvc_score_test=qsvc.score(data_test, data_test_labels )
print(f"QSVC classification test score PauliFeatureMap: {qsvc_score_test}")
#print(f"Time taken :{t1-t0}")


qsvc_score_train=qsvc.score(data_train, data_train_labels)
print(f"QSVC classification train score Zzfeaturemap: {qsvc_score_train}")

<ipython-input-10-312ee3e1dc4b>:18: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()


2
Time taken for QSVC training PauliFeatureMap: 118.81094670295715 seconds
QSVC classification test score PauliFeatureMap: 0.9
QSVC classification train score Zzfeaturemap: 0.93125
